In [2]:
import tequila as tq
from lcu_v1 import LCU

# Introduction

In this report, we focus on several practical applications of the LCU algorithm, which we have discussed in Reports 1 through 3. While we will not be delving into the exact details of the code to the extent that was seen in the previous reports, we hope that the use of the LCU class, as introduced in Report 3, shall prove to be a helpful abstraction into implementing the following applications.

As an additional disclaimer, the applications discussed here claim no result for optimality, and there have been multiple algorithms which prove to be more efficient than applying the LCU method. However, with such a wide range of applications, we wish to show proof-of-concept that the LCU algorithm comes with helpful generalizations which can be used to abstract away some of the technicalities involved in defining circuits for certain operations.

# Goals of this report: 

TODO

# Motivation

TODO

Block-encoding

# Application 1: Implementing projectors

TODO

# Application 2: Implementing powers of operators

## Introduction and problem setting

Often, implementing some Hamiltonian, $H$, for some type of physical systems is not particularly scaleable due to having a large polynomial complexity, or worse. As such, implementing powers of $H$, say $H^k$ where $k\in\mathbb{N}$, further worsens this complexity. For instance, suppose that implementing $H$ requires a total of $N$ terms where $N$ is some expression in terms of the input; then, implementing $H^k$ will require $N^k$ terms, which scales exponentially with the power required.

For this problem, it suffices to consider implementing $H^2$ instead of $H^k$ for some general $k$, since the extension is fairly straightforward.

## Applying LCU

Note that, using the LCU method, we can simply place two of the LCU blocks for $H$ to approximate the effect of $H^2$.

The success probability of doing so relies on measuring the ancilla for both LCU blocks in the zero state. For the first LCU block, after going through the amplitude amplification routine, will be approximately $\left|\left| H \left| \psi \right\rangle \right| \right| ^2$, where $\left| \psi \right\rangle$ was the initial state. Similarly, the success probability for the second LCU block will be $\left|\left| H^2 \left| \psi \right\rangle \right| \right| ^2$.

Combining the two LCU blocks, we see that the success probability of such an implementation of $H^2$ is $\left|\left| H \left| \psi \right\rangle \right| \right| ^2 \cdot \left|\left| H^2 \left| \psi \right\rangle \right| \right| ^2$.

More generally, with some initial state $\left| \psi \right\rangle$, the success probability of such an implementation of $H^k$, for any $k\in\mathbb{N}$ will be $\prod_{j=1}^k \left|\left| H^k \left| \psi \right\rangle \right| \right| ^2$.

### Note on reusing ancilla qubits

For the general case, we cannot use the ancilla qubits in more than one LCU block in the above implementation. This is because we are unable to deterministically improve the success probability of the LCU method to 1, and so, there remains some non-negligible probability that the ancilla is not in the zero-state as required. Therefore, reusing ancilla qubits in such a generalized implementation introduces certain dependencies of the later LCU blocks on the earlier blocks, which worsens our overall success probability.

Hence, to implement $H^k$, where $H$ is written as a linear combination of $m$ unitary operations, we shall require $\mathcal{O}(\log m)$ ancilla qubits for each LCU block, for a total of $k$ LCU blocks. Thus, the total number of ancilla qubits required for such an implementation is in $\mathcal{O}(k \log m)$.

However, now suppose that $H$ itself is unitary, or is at least 'close' to being unitary in the sense that there exists some unitary $U$ and some small $\epsilon > 0$ such that $\left| \left| H - U \right| \right| < \epsilon$. Then, we are guaranteed success through the oblivious amplitude amplification procedure at the end of each LCU block. This implies that, after measuring, the probability of the ancilla not being in the zero state is very negligible and thus, we can resuse the ancilla qubits repeatedly. Hence, the number of ancilla qubits in such a case is in $\mathcal{O}(\log m)$.

## Implementing in code

TODO

## Uses and extensions
TODO

Implementing polynomials in $H$ of finite degree $n$.

One natural extension of this technique is to consider implementing some arbitrary polynomial $P$ of $H$ with degree $n\in\mathbb{N}$, $P = \sum_{j\in[n]} \alpha_j H^j$. Since we have expressed each of the ...

Use: expectation value and excited state optimization

# Application 3: Time evolution using truncated Taylor series

## Introduction

Suppose that we wish to simulate the evolution under some Hamiltonian $H$ for time $t$ within a maximum error of some $\epsilon > 0$ with the operator $U = e^{-iHt}$. For this example, we further make the assumption that $H$ is unitary.

Divide the time $t$ into $r$ equal time-segments of length $t/r$. Further suppose that we can write $H$ as a linear combination $H = \sum_{j=0}^m \alpha_j H_j$, where each $H_j$ is unitary and $\alpha_j > 0$ for all $j$. For each segment, define the operator for time-evolution within that segment as follows:
\begin{align*}
    U_r = e^{-iHt/r} \approx \sum_{j=0}^K \frac1{j!} (-iHt/r)^j
\end{align*}

Here, we wish to truncate the Taylor series expansion of the exponential at some order $K$ such that the maximum error for each segment is $\epsilon/r$, which would imply that the maximum result across all the segments is $\epsilon$. Assuming that $r > \left|\left| H \right|\right| t$ (where $\left|\left| \cdot \right|\right|$ denotes the usual matrix norm), it has been shown in (INSERT CITATION) that we must have:
\begin{align*}
    K &\in \mathcal{O}\left( \frac{\log(r/\epsilon)}{\log(\log(r/\epsilon))} \right)
\end{align*}

Therefore, we have that:
\begin{align*}
    U &= e^{-iHt} \\
    &= \prod_{j=0}^{r-1} U_r \tag{$U_r = e^{-iHt/r}$} \\
    &\approx \prod_{j=0}^{r-1} \sum_{k=0}^K \frac1{k!} (-iHt/r)^k \\
    &= \prod_{j=0}^{r-1} \sum_{k=0}^K \frac{(-it/r)^k}{k!} H^k \\
\end{align*}

## Applying LCU

We first note that, in our implementation of the LCU algorithm, the only constraint on the operator $H$ was that we assumed $H$ to be Hermitian. This implies that we can think of $H$ as a Hamiltonian, which we shall now do so.

Now, since we already know how to implement $H$ using the LCU algorithm explained previously, this implies that we can also implement all powers of $H$, $H^k$, by simply repeating the circuit for $H$. Furthermore, since the LCU circuit for $H$ was unitary, this implies that all powers of this circuit will also be unitary. This is indeed nice, as it is now similar to our LCU framework.

Thus, to construct an implementation of $U_r$, all we need to do is to apply a $(-i)^k$ phase to the circuit for $H^k$. This can be done in Tequila by using generators, as follows:

TODO

## Implementing in code

TODO

## Further extensions

TODO: Talk about potential application to HHL algorithm, which relies on Hamiltonian simulation to solve linear systems of equations.

# Application 4: SWAP Test and Hadamard Test

TODO

# Application 5:

TODO

# References

TODO